# Named Entity Recognition using Language Models
This lab will focus on helping you fine-tune Transformers models for Named Entity Recognition. You will use the Datasets library to quickly load and preprocess the WikiNEuRal dataset, getting them ready for training with PyTorch.

#### By: Nazia Nafis
---
---

# 2. Using XLM-R

## Installation
Let's first install the required libraries

In [40]:
!pip install datasets transformers seqeval huggingface_hub

In [41]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!sudo apt-get install git-lfs
!git lfs install

Detected operating system as Ubuntu/bionic.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... done.
Running apt-get update... done.

The repository is setup! You can now install packages.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (3.1.2).
0 upgraded, 0 newly installed, 0 to remove and 97 not upgraded.
Git LFS initialized.


## Dataset(s)

We will use the Datasets library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be done with the functions load_dataset and load_metric.

The notebook should work with any token classification dataset provided by the Datasets library.

In [42]:
from datasets import load_dataset, load_metric, concatenate_datasets
datasets = load_dataset("Babelscape/wikineural")

Using custom data configuration Babelscape--wikineural-2c05bc228ce4015c
Reusing dataset parquet (/root/.cache/huggingface/datasets/parquet/Babelscape--wikineural-2c05bc228ce4015c/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901)


  0%|          | 0/27 [00:00<?, ?it/s]

The WikiNEuRal dataset has train, validation and test splits for each of the 9 languages.

To access an actual element, we need to select a split first, then give an index:

In [43]:
datasets["train_en"][0]

{'lang': 'en',
 'ner_tags': [0, 0, 0, 0, 0, 5, 6, 0, 0, 0, 0, 5, 0, 0],
 'tokens': ['This',
  'division',
  'also',
  'contains',
  'the',
  'Ventana',
  'Wilderness',
  ',',
  'home',
  'to',
  'the',
  'California',
  'condor',
  '.']}

The labels are already coded as integer ids to be easily usable by our model, but the correspondence with the actual categories is stored the following:

In [44]:
label_list = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

labels_vocab = {'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}
labels_vocab_reverse = {v:k for k,v in labels_vocab.items()}

If we want to train a monolingual model (NER on a single language), we have to use a single language for each split.

In [45]:
train_dataset = datasets["train_en"]
val_dataset = datasets["val_en"]
test_dataset = datasets["test_en"]

This is how we can concatenate multiple dataset splits to train a Multilingual NER model.

In [46]:
train_dataset = concatenate_datasets([datasets["train_de"], 
                                       datasets["train_en"], 
                                       datasets["train_es"]])

val_dataset = concatenate_datasets([datasets["val_de"], 
                                     datasets["val_en"], 
                                     datasets["val_es"]])

test_dataset = concatenate_datasets([datasets["test_de"], 
                                     datasets["test_en"], 
                                    datasets["test_es"]])

## Pre-processing

Before we can feed those texts to our model, we need to preprocess them. This is done by a Transformers Tokenizer which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the AutoTokenizer.from_pretrained method, which will ensure:

 - we get a tokenizer that corresponds to the model architecture we want to use,
 - we download the vocabulary used when pretraining this specific checkpoint.
 
That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [47]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base") ## We use the BERT tokenizer

Could not locate the tokenizer configuration file, will try to use the model config instead.
https://huggingface.co/xlm-roberta-base/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpm9y9zvuq


Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

storing https://huggingface.co/xlm-roberta-base/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/87683eb92ea383b0475fecf99970e950a03c9ff5e51648d6eee56fb754612465.dfaaaedc7c1c475302398f09706cbb21e23951b73c6e2b3162c1c8a99bb3b62a
creating metadata file for /root/.cache/huggingface/transformers/87683eb92ea383b0475fecf99970e950a03c9ff5e51648d6eee56fb754612465.dfaaaedc7c1c475302398f09706cbb21e23951b73c6e2b3162c1c8a99bb3b62a
loading configuration file https://huggingface.co/xlm-roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/87683eb92ea383b0475fecf99970e950a03c9ff5e51648d6eee56fb754612465.dfaaaedc7c1c475302398f09706cbb21e23951b73c6e2b3162c1c8a99bb3b62a
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

storing https://huggingface.co/xlm-roberta-base/resolve/main/sentencepiece.bpe.model in cache at /root/.cache/huggingface/transformers/9df9ae4442348b73950203b63d1b8ed2d18eba68921872aee0c3a9d05b9673c6.00628a9eeb8baf4080d44a0abe9fe8057893de20c7cb6e6423cddbf452f7d4d8
creating metadata file for /root/.cache/huggingface/transformers/9df9ae4442348b73950203b63d1b8ed2d18eba68921872aee0c3a9d05b9673c6.00628a9eeb8baf4080d44a0abe9fe8057893de20c7cb6e6423cddbf452f7d4d8
https://huggingface.co/xlm-roberta-base/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp5dg3ekho


Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

storing https://huggingface.co/xlm-roberta-base/resolve/main/tokenizer.json in cache at /root/.cache/huggingface/transformers/daeda8d936162ca65fe6dd158ecce1d8cb56c17d89b78ab86be1558eaef1d76a.a984cf52fc87644bd4a2165f1e07e0ac880272c1e82d648b4674907056912bd7
creating metadata file for /root/.cache/huggingface/transformers/daeda8d936162ca65fe6dd158ecce1d8cb56c17d89b78ab86be1558eaef1d76a.a984cf52fc87644bd4a2165f1e07e0ac880272c1e82d648b4674907056912bd7
loading file https://huggingface.co/xlm-roberta-base/resolve/main/sentencepiece.bpe.model from cache at /root/.cache/huggingface/transformers/9df9ae4442348b73950203b63d1b8ed2d18eba68921872aee0c3a9d05b9673c6.00628a9eeb8baf4080d44a0abe9fe8057893de20c7cb6e6423cddbf452f7d4d8
loading file https://huggingface.co/xlm-roberta-base/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/daeda8d936162ca65fe6dd158ecce1d8cb56c17d89b78ab86be1558eaef1d76a.a984cf52fc87644bd4a2165f1e07e0ac880272c1e82d648b4674907056912bd7
loading file h

The following assertion ensures that our tokenizer is a fast tokenizers (backed by Rust) from the Tokenizers library. Those fast tokenizers are available for almost all models, and we will need some of the special features they have for our preprocessing.

In [48]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

We can directly call this tokenizer on one sentence, or we can use the tokenize method to tokenize multiple sentences at once.

In [49]:
tokenizer("Hello, this is one sentence!")

{'input_ids': [0, 35378, 4, 903, 83, 1632, 149357, 38, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

This is a good overal tutorial for preprocessing for Transformer. Check this out to learn more about the preprocessing steps.
https://huggingface.co/transformers/preprocessing.html

Here we set the labels of all special tokens to -100 (the index that is ignored by PyTorch) and the labels of all other tokens to the label of the word they come from. Another strategy is to set the label only on the first token obtained from a given word, and give a label of -100 to the other subtokens from the same word. Just change the value of the following flag:

In [50]:
label_all_tokens = False

We're now ready to write the function that will preprocess our samples. We feed them to the tokenizer with the argument truncation=True (to truncate texts that are bigger than the maximum size allowed by the model) and is_split_into_words=True (as seen above). Then we align the labels with the token ids using the strategy we picked:

In [51]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the map method of our dataset object we created earlier. This will apply the function on all the elements of all the splits in dataset, so our training, validation and testing data will be preprocessed in one single command.

In [52]:
train_tokenized = train_dataset.map(tokenize_and_align_labels, batched=True)
val_tokenized = val_dataset.map(tokenize_and_align_labels, batched=True)
test_tokenized = test_dataset.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/268 [00:00<?, ?ba/s]

  0%|          | 0/34 [00:00<?, ?ba/s]

  0%|          | 0/34 [00:00<?, ?ba/s]

Even better, the results are automatically cached by the Datasets library to avoid spending time on this step the next time you run your notebook. The Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. Datasets warns you when it uses cached files, you can pass load_from_cache_file=False in the call to map to not use the cached files and force the preprocessing to be applied again.

Note that we passed batched=True to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning

In [53]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained("xlm-roberta-base", num_labels=len(label_list), label2id=labels_vocab, id2label=labels_vocab_reverse)

loading configuration file https://huggingface.co/xlm-roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/87683eb92ea383b0475fecf99970e950a03c9ff5e51648d6eee56fb754612465.dfaaaedc7c1c475302398f09706cbb21e23951b73c6e2b3162c1c8a99bb3b62a
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "B-PER",
    "2": "I-PER",
    "3": "B-ORG",
    "4": "I-ORG",
    "5": "B-LOC",
    "6": "I-LOC",
    "7": "B-MISC",
    "8": "I-MISC"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B-LOC": 5,
    "B-MISC": 7,
    "B-ORG": 3,
    "B-PER": 1,
    "I-LOC": 6,
    "I-MISC": 8,
    "I-ORG": 4,
    "I-PER": 2,
    "O": 0
  },
  "layer_norm_

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

storing https://huggingface.co/xlm-roberta-base/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/97d0ea09f8074264957d062ec20ccb79af7b917d091add8261b26874daf51b5d.f42212747c1c27fcebaa0a89e2a83c38c6d3d4340f21922f892b88d882146ac2
creating metadata file for /root/.cache/huggingface/transformers/97d0ea09f8074264957d062ec20ccb79af7b917d091add8261b26874daf51b5d.f42212747c1c27fcebaa0a89e2a83c38c6d3d4340f21922f892b88d882146ac2
loading weights file https://huggingface.co/xlm-roberta-base/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/97d0ea09f8074264957d062ec20ccb79af7b917d091add8261b26874daf51b5d.f42212747c1c27fcebaa0a89e2a83c38c6d3d4340f21922f892b88d882146ac2
Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForTokenClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight']
- Th

The warning is telling us we are throwing away some weights (the vocab_transform and vocab_layer_norm layers) and randomly initializing some other (the pre_classifier and classifier layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate a Trainer, we will need to define three more things. The most important is the TrainingArguments, which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [54]:
model_name = "xlm-roberta-base" ## You Change this to any generic language model from HuggingFace which supports fine-tuning.
args = TrainingArguments(                   ## This is where all your hyperparameters are, reduce *_batch_size if you are working with a GPU which has limited memory.
    "wikineural-multilingual-ner",
    evaluation_strategy = "steps",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,                     ## may not be enough for a large data, increase this to 5 later in your experiments.
    weight_decay=0.01,
    push_to_hub=True,
    eval_steps=10000,
    save_steps=10000,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Here we set the evaluation to be done at the end of each epoch, tweak the learning rate,customize the batch_size and the number of epochs for training, as well as the weight decay.

Then we will need a data collator that will batch our processed examples together while applying padding to make them all the same size (each pad will be padded to the length of its longest example). There is a data collator for this task in the Transformers library, that not only pads the inputs, but also the labels:

In [55]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

The last thing to define for our Trainer is how to compute the metrics from the predictions. Here we will load the seqeval metric (which is commonly used to evaluate results on the CONLL dataset) via the Datasets library.

In [56]:
metric = load_metric("seqeval")

So we will need to do a bit of post-processing on our predictions:

 - select the predicted index (with the maximum logit) for each token
 - convert it to its string label
 - ignore everywhere we set a label of -100
 
The following function does all this post-processing on the result of Trainer.evaluate (which is a namedtuple containing predictions and labels) before applying the metric:

In [57]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

Before you go any further:
1. Login to the HuggigFace CLI on your system using command ``huggingface-cli login``
2. Get your access token from your user account on HuggingFace at https://huggingface.co/settings/tokens (preferably a write access token)
3. Use this token with the command above to ensure being logged in to HuggingFace.

IF you are not working on your own machine with a GPU and have opened this notebook in Colab / Studio Lab, you can use the also cell below to login. You could have used this cell instead of commandline (cli) login, but where is the fun in that?

In [58]:
from huggingface_hub import notebook_login
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


Note that we drop the precision/recall/f1 computed for each category and only focus on the overall precision/recall/f1/accuracy.

Then we just need to pass all of this along with our datasets to the Trainer:

In [59]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/content/wikineural-multilingual-ner is already a clone of https://huggingface.co/nn007/wikineural-multilingual-ner. Make sure you pull the latest changes with `repo.git_pull()`.


In [60]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [61]:
! pip install torchvision 
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

In [62]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `XLMRobertaForTokenClassification.forward` and have been ignored: ner_tags, lang, tokens. If ner_tags, lang, tokens are not expected by `XLMRobertaForTokenClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 267680
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 16730


Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
10000,0.024900,0.023074,0.941555,0.945696,0.943621,0.992893


The following columns in the evaluation set  don't have a corresponding argument in `XLMRobertaForTokenClassification.forward` and have been ignored: ner_tags, lang, tokens. If ner_tags, lang, tokens are not expected by `XLMRobertaForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 33587
  Batch size = 16
Saving model checkpoint to wikineural-multilingual-ner/checkpoint-10000
Configuration saved in wikineural-multilingual-ner/checkpoint-10000/config.json
Model weights saved in wikineural-multilingual-ner/checkpoint-10000/pytorch_model.bin
tokenizer config file saved in wikineural-multilingual-ner/checkpoint-10000/tokenizer_config.json
Special tokens file saved in wikineural-multilingual-ner/checkpoint-10000/special_tokens_map.json
tokenizer config file saved in wikineural-multilingual-ner/tokenizer_config.json
Special tokens file saved in wikineural-multilingual-ner/special_tokens_map.json
Adding files tracked by Git LFS

TrainOutput(global_step=16730, training_loss=0.03354847084884507, metrics={'train_runtime': 9486.1261, 'train_samples_per_second': 28.218, 'train_steps_per_second': 1.764, 'total_flos': 9329521522390848.0, 'train_loss': 0.03354847084884507, 'epoch': 1.0})

## Evaluation

The evaluate method allows us to evaluate again on the evaluation dataset or on another dataset:

In [63]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `XLMRobertaForTokenClassification.forward` and have been ignored: ner_tags, lang, tokens. If ner_tags, lang, tokens are not expected by `XLMRobertaForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 33587
  Batch size = 16


{'epoch': 1.0,
 'eval_accuracy': 0.9937662813129251,
 'eval_f1': 0.9503634948655165,
 'eval_loss': 0.019714558497071266,
 'eval_precision': 0.9474142335421122,
 'eval_recall': 0.9533311753841467,
 'eval_runtime': 304.3038,
 'eval_samples_per_second': 110.373,
 'eval_steps_per_second': 6.901}

To get the precision/recall/f1 computed for each category now that we have finished training, we can apply the same function as before on the result of the predict method:

In [64]:
predictions, labels, _ = trainer.predict(test_tokenized)
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

The following columns in the test set  don't have a corresponding argument in `XLMRobertaForTokenClassification.forward` and have been ignored: ner_tags, lang, tokens. If ner_tags, lang, tokens are not expected by `XLMRobertaForTokenClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 33587
  Batch size = 16


{'LOC': {'f1': 0.9626963925217239,
  'number': 17112,
  'precision': 0.9639655475478994,
  'recall': 0.9614305750350631},
 'MISC': {'f1': 0.8928150765606596,
  'number': 10516,
  'precision': 0.884769819777757,
  'recall': 0.9010079878280715},
 'ORG': {'f1': 0.9483897605284889,
  'number': 9639,
  'precision': 0.9436171305330184,
  'recall': 0.9532109139952277},
 'PER': {'f1': 0.9777676060942914,
  'number': 15252,
  'precision': 0.9751532542063388,
  'recall': 0.9803960136375557},
 'overall_accuracy': 0.9937662813129251,
 'overall_f1': 0.9503634948655165,
 'overall_precision': 0.9474142335421122,
 'overall_recall': 0.9533311753841467}

---
---
In this notebook, we saw the performance of **XLM Roberta Base** model.

We got the following metrics overall:

- Accuracy: 0.99
- F1: 0.95
- Precision: 0.95
- Recall: 0.95